In [ ]:
import cv2
import numpy as np
import os
import time

# Mematikan log pesan dari TensorFlow agar terminal lebih bersih
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array

# ==========================================
# 1. KONFIGURASI
# ==========================================
YOLO_CFG = "yolov3-tiny.cfg"      
YOLO_WEIGHTS = "yolov3-tiny.weights"
COCO_NAMES = "coco.names"
CNN_MODEL = "vehicle_classifier.h5"
LABELS = ["Bikes", "Cars", "Motorcycles", "Trains", "Auto Rickshaws"]

CONF_THRESH = 0.4
NMS_THRESH = 0.35
DETECT_INTERVAL = 5 
RECLASS_INTERVAL = 15

# ==========================================
# 2. FUNGSI PENDUKUNG
# ==========================================
def create_tracker():
    try:
        return cv2.TrackerKCF_create()
    except AttributeError:
        return cv2.legacy.TrackerKCF_create()

def preprocess_for_cnn(roi):
    img = cv2.resize(roi, (224, 224))
    img = img.astype("float32") / 255.0
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    return img

# ==========================================
# 3. MEMUAT MODEL AI
# ==========================================
print("[INFO] Memuat model AI, mohon tunggu...")
net = cv2.dnn.readNet(YOLO_WEIGHTS, YOLO_CFG)
with open(COCO_NAMES, "r") as f:
    classes = [c.strip() for c in f.readlines()]

vehicle_classes = {"car", "motorbike", "bus", "truck", "bicycle", "train"}
model = load_model(CNN_MODEL)

# ==========================================
# 4. KERANGKA UTAMA DENGAN PENGAMAN (TRY...FINALLY)
# ==========================================
cap = cv2.VideoCapture(0)
layer_names = net.getLayerNames()
out_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers().flatten()]

trackers = []
frame_count = 0
t0 = time.time()

try:
    print("[INFO] Kamera aktif. Klik jendela kamera lalu tekan 'q' untuk berhenti.")
    
    while True:
        ret, frame = cap.read()
        if not ret: break
        
        frame_count += 1
        h, w = frame.shape[:2]

        # PROSES DETEKSI YOLO
        if frame_count % DETECT_INTERVAL == 1 or len(trackers) == 0:
            blob = cv2.dnn.blobFromImage(frame, 1/255.0, (320, 320), swapRB=True, crop=False)
            net.setInput(blob)
            outs = net.forward(out_layers)
            
            boxes, confs = [], []
            for out in outs:
                for det in out:
                    scores = det[5:]
                    class_id = np.argmax(scores)
                    if classes[class_id] in vehicle_classes and scores[class_id] > CONF_THRESH:
                        bw, bh = int(det[2] * w), int(det[3] * h)
                        x, y = int((det[0] * w) - bw / 2), int((det[1] * h) - bh / 2)
                        boxes.append([x, y, bw, bh])
                        confs.append(float(scores[class_id]))
            
            indices = cv2.dnn.NMSBoxes(boxes, confs, CONF_THRESH, NMS_THRESH)
            trackers = []
            if len(indices) > 0:
                for i in indices.flatten():
                    x, y, bw, bh = boxes[i]
                    x, y = max(0, x), max(0, y)
                    bw, bh = min(bw, w - x), min(bh, h - y)
                    tr = create_tracker()
                    tr.init(frame, (x, y, bw, bh))
                    
                    label, score = "Checking...", 0
                    roi = frame[y:y+bh, x:x+bw]
                    if roi.size > 0:
                        p_img = preprocess_for_cnn(roi)
                        pred = model.predict(p_img, verbose=0)[0]
                        label = LABELS[np.argmax(pred)]
                        score = np.max(pred) * 100
                    
                    trackers.append({'tr': tr, 'box': (x,y,bw,bh), 'label': label, 'score': score})
        
        # UPDATE POSISI (TRACKING)
        else:
            new_trackers = []
            for t_data in trackers:
                success, box = t_data['tr'].update(frame)
                if success:
                    tx, ty, tw, th = [int(v) for v in box]
                    tx, ty = max(0, tx), max(0, ty)
                    tw, th = min(tw, w - tx), min(th, h - ty)
                    t_data['box'] = (tx, ty, tw, th)
                    new_trackers.append(t_data)
            trackers = new_trackers

        # GAMBAR KE LAYAR
        for t_data in trackers:
            x, y, bw, bh = t_data['box']
            cv2.rectangle(frame, (x, y), (x+bw, y+bh), (0, 255, 0), 2)
            cv2.putText(frame, f"{t_data['label']}", (x, y-10), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        cv2.imshow("Deteksi Kendaraan", frame)
        
        # Tekan 'q' untuk keluar
        if cv2.waitKey(1) & 0xFF == ord('q'):
            print("[INFO] Berhenti karena tombol 'q' ditekan.")
            break

except Exception as e:
    print(f"[ERROR] Terjadi kesalahan: {e}")

finally:
    # AKAN SELALU DIJALANKAN
    print("[INFO] Menutup kamera dan membersihkan memori...")
    cap.release()
    cv2.destroyAllWindows()
    for i in range(5): # Memastikan jendela benar-benar tertutup di Jupyter
        cv2.waitKey(1)